# Introduction to the MIMIC-IV Database and Machine Learning with scikit-learn

**Author: Meghan R. Hutch**

**Date: January 30th, 2025**

### **About Me**

* Current Preceptor of Data Science at the University of Chicago
* Past PhD Candidate the Luo Lab 
* Dissertation focused on using EHR data to develop predictive models for patients with neurological illness
* Graduated summer 2024 from DGP, Biomedical Informatics Concentration

### **Today's lecture will be divided into two parts:**

* Introduction to working with the MIMIC-IV dataset

* Introduction to machine learning with scikit learn


We will discuss both of these in the context of a framework for conducting clinical research

**Inspired and adapted from Dr. Garrett Eickelberg's [workshop on MIMIC-III and scikit learn](https://github.com/geickelb/MIMIC-III_to_Model/tree/master)*

# Framework for Conducting Clinical Research 

---

1. Research Question Specification (hypothesis)
<br></br>
2. Cohort Specification
<br></br>
3. Data Extraction
<br></br>
4. Data Pre-processing (cleaning!)
<br></br>
5. Model Training
<br></br>
5. Model Evaluation
<br></br>

**Today, we will largely focus on steps 3-6**

### Introducing the MIMIC-IV Database

* The Medical Information Mart for Intensive Care (MIMIC)-IV is a large dataset curated to help support studies on intensive care unit (ICU) patients. 

* The MIMIC-IV dataset (since April 2024) contains structured EHR data from >360,000 patients who were admitted to Beth Israel Deaconess Medical Center in Boston, MA between 2008-2022.

* Today, we will use a previously curated [MIMIC-IV demo-dataset](https://physionet.org/content/mimic-iv-demo/2.2/) to introduce how EHR data is computationally structured and how it can be used for research purposes.

* Notablly, this demo-dataset contains the data from a random subset of 100 hospitalized patients. 

* This dataset was curated for the purposes of workshops and tutorials. Thus, having appropriate CITI training is not needed. Although deidentified, access to the full dataset requires a physionet account and [CITI training](https://physionet.org/about/citi-course/).

# Data Extraction

- The full demo dataset has been downloaded to the classroom's quest allocation.
- The data used for this lecture are available in the relevant [GitHub Repo](https://github.com/meghutch/mimic-ml-demo)
- We will further load this data into our notebook


### Load in MIMIC-IV Demo

First, we will import some basic packages

In [ ]:
# import our packages
import os

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# set our working directory - this notebook assume you are in the classroom Quest folder ('/projects/e30766/mimic-iv-demo/2.2') where the MIMIC-IV demo dataset is located
#os.chdir('/projects/e30766/mimic-iv-demo/2.2')

In [ ]:
# set relative working directory assuming you downloaded this lecture from GitHub:
os.chdir('data/mimic-iv-clinical-database-demo-2.2')

In [ ]:
# make sure you are in the working directory: 
os.getcwd()

Next, we will load the list of subject identifiers (`subject_id`) for the demo cohort

In [ ]:
# load data
subject_id = pd.read_csv('demo_subject_id.csv')

In [ ]:
# view the first 5 rows of data
subject_id.head(5)

In [ ]:
# check how many unique subjects
subject_id['subject_id'].nunique()

**Overview of MIMIC-IV Tables**

For this demo, we will explore data found in the following two MIMIC-IV modules:


### [Hosp](https://mimic.mit.edu/docs/iv/modules/hosp/)

> "The Hosp module provides all data acquired from the hospital wide electronic health record. Information covered includes patient and admission information, laboratory measurements, microbiology, medication administration, and billed diagnoses."

### [ICU](https://mimic.mit.edu/docs/iv/modules/icu/)

> "The ICU module contains information collected from the clinical information system used within the ICU. Documented data includes intravenous administrations, ventilator settings, and other charted items."

**Notes:**

* The [GitHub repo](https://github.com/meghutch/mimic-ml-demo/tree/main) for this lecture only contains the data used for this workshop (due to GitHub storage limits). However, all data tables within the `hosp` and `icu` modules have been downloaded to Quest: `/projects/e30766/mimic-iv-demo/2.2`

* The full MIMIC-IV dataset also contains additional modules for data from emergency department (ED), chest X-rays (CXR), and clinical notes (Note)

Next, let's look at the `patient` table found within the `hosp` module.

The `patient` table contains each patient's unique `subject_id` and some simple demographic data

In [ ]:
patients = pd.read_csv('hosp/patients.csv.gz')

In [ ]:
# review the first 5 rows
patients.head()

What do these columns mean? 

For this, we can review the data dictionary: https://mimic.mit.edu/docs/iv/modules/hosp/patients/

In [ ]:
# review the first 5 rows
patients.head()

Important note about MIMIC's pre-processing of [dates](https://mimic.mit.edu/docs/iv/modules/hosp/patients/#anchor_age-anchor_year-anchor_year_group):

* Dates are randomly shifted consistently for all patients

* `anchor_year` - shifted year for the patient

* `anchor_year_group` -  is a range of years - the patient’s `anchor_year` actually occurred during this range

* `anchor_age` - the patient’s age in the `anchor_year` (year of admission). 

**Examples**: 

A patient has an `anchor_year` of 2153, `anchor_year_group` of 2008 - 2010, and an `anchor_age` of 60.

* The year 2153 for the patient corresponds to 2008, 2009, or 2010.

* The patient was 60 in the shifted year of 2153, i.e. they were 60 in 2008, 2009, or 2010.

* A patient admission in 2154 will occur in 2009-2011, an admission in 2155 will occur in 2010-2012, and so on.

* Note: Due to HIPPA patient confidentiality concerns, if a patient’s `anchor_age` is over 89 in the `anchor_year` then their `anchor_age` is set to 91, regardless of how old they actually were.

In [ ]:
# review the first 5 rows
patients.head()

In [ ]:
# review the number of unique values in each dataset
patients['subject_id'].nunique()

In [ ]:
len(patients)

**Let's load in several additional tables from MIMIC:**

Here, we will load a few of the tables we will use to demonstate the workflow of aquiring, pre-processing, and analyzing data from the MIMIC database.

In [ ]:
# load several tables from the hosp module
admissions = pd.read_csv('hosp/admissions.csv.gz') # info about hospital stay
omr = pd.read_csv('hosp/omr.csv.gz') # misc info from EHR
labevents = pd.read_csv('hosp/labevents.csv.gz') # laboratory measurements
d_labitems = pd.read_csv('hosp/d_labitems.csv.gz') # dictionary of laboratory values
diagnoses = pd.read_csv('hosp/diagnoses_icd.csv.gz') # icd codes assigned to patient for billing
d_icd_diagnoses = pd.read_csv('hosp/d_icd_diagnoses.csv.gz') # dictionary of icd variable

# load several tables from the icu module
chartevents = pd.read_csv('icu/chartevents.csv.gz') # info charted during patient's hospital stay
d_items = pd.read_csv('icu/d_items.csv.gz') # dictionary of ICU event variables  

### omr table

> "The Online Medical Record (OMR) table contains miscellaneous information from the EHR."

[omr documentation](https://mimic.mit.edu/docs/iv/modules/hosp/omr) of columns:

* `chartdate` - the date on which the observation was recorded
* `seq_num`- a monotonically increasing integer which uniquely distinguishes results of the same type recorded on the same day. For example, if two blood pressure measurements occur on the same day, seq_num orders them chronologically.
* `result_name` - human interpretable description of the observation
* `result_value` - the avalue associated with the observation

In [ ]:
# review the first 5 rows
omr.head()

In [ ]:
# review unique result_names (observations)
# here we can see the unique variables/measurements contained in this table
omr[['result_name']].drop_duplicates()

# Data Pre-processing (Data Cleaning)

### What does it mean to clean data?

Simply, data cleaning is the act of preparing data for analysis. In a clinical context, we also want to make sure that the data we are using is clinically relevant and accurate, especially in regard to the specific clinical question or problem we are trying to solve.


### How do we "clean" data:

* Ensure data are formatted into the right data types (numeric, character, date, etc)

* Review summary statistics (how many observations, how many patients have each observation, mean/median/sd/variance of values, proportion of missing data)

* Review the distribution of data

* Are there possible data entry problems or unit conversions needed?

* How should we handle missing data?

* Should patients have repeated measurements (e.g. multiple white blood cell counts on the same day)? If so, how should we handle repeated observations?

> “Data science, he says, involves multiple “very small decisions” — data cleaning and filtering steps, for instance, which are crucially important, but difficult to document. And journal page limits preclude exposition. But by blending code, data and text in a single document, researchers can show just how their results were generated.” - Ben Marwick; [Perkel, JM](https://www.nature.com/articles/d41586-022-00563-z)

### Example of Data Cleaning (Height)

**Let's focus on the variables `Height` and `Weight`**

We will filter the `omr` table to only contains results for height and weight. During data cleaning, each variable will be saved in its own dataframe for ease of pre-processing

In [ ]:
# create separate dataframes for each variable
height = omr[omr['result_name']=="Height (Inches)"].copy() # copy tells Python to keep our new df 'height' independent from the original df 'omr' it was sourced from

weight = omr[omr['result_name']=="Weight (Lbs)"].copy()

**Examine Height (inches)**

In [ ]:
height.head()

Let's check the number of observations

In [ ]:
print('Number of height measurements:', len(height))
print('Number of unique patients with height measurements:', height['subject_id'].nunique())

It looks like `result_value` is a continuous variable. Let's make sure that Python also has recognized it as such.

In [ ]:
# check variable data types
height.dtypes

It looks like Python read `result_value` as a object (or character) variable. Let's convert to numeric. This is an important step to make sure any downstream functions (calculating the mean or median, plotting the distribution, etc) work correctly.

In [ ]:
height['result_value'] = pd.to_numeric(height['result_value'])

In [ ]:
# check variable data types again to make sure our transformation and code worked
# float64 indicates a continuous variable, so this looks good!
height.dtypes

### Using Summary Statistics and Visualization to help Evaluate Data

Next, let's calculate summary statistics to get a better sense of our variable

In [ ]:
height['result_value'].describe()

**Visualize distribution of values**

In [ ]:
sns.displot(height['result_value'])

**Potential red flag:** Is there really a patient who might be 5 inches tall? This is a clear case I'd like to investigate more. Perhaps this is a data entry issue or a newborn?

**Further explore data to check for inconsistencies**

First, we can sort the height values from lowest to highest in order to retrieve the `subject_id` of the patient with a height of 5 inches

I can also see if there are any other patients with unusually low heights

In [ ]:
height.sort_values(['result_value'])

Let's see if this patient has other recorded height measurements

In [ ]:
# let's see if this patient has other recorded height measurements
height[height['subject_id']==10012853].sort_values('chartdate')

**Investigate patients with multiple observations**

For each unique patient, we will calculate the standard deviation of the patient's height measurements

*Note*: `ddof = 0` indicates that std for patients with one measurement will be displayed as 0 rather than NaN

In [ ]:
height['std'] = height.groupby('subject_id')['result_value'].transform(np.std, ddof=0)

In [ ]:
height.sort_values('std', ascending = False)[['subject_id', 'std']].drop_duplicates().head(10)

We can clearly see the one 'concerning' patient has a much higher standard deviation of their height measurements compared to other patients.

Let's take a look at patient with the second highest standard deviation 

In [ ]:
height[height['subject_id']==10005909].sort_values('chartdate')

**For patients with multiple observations, let's calculate the median**

In [ ]:
# create a new variable with the median height value for each patient
height['result_value_median'] = height.groupby('subject_id')['result_value'].transform('median')

**Save the new dataset**

This next step ensures that we will now only have one unique row per patient

In [ ]:
# assuming we don't need to link by date
height_clean = height[['subject_id', 'result_name', 'result_value_median']].drop_duplicates()

Let's review the distribution again

In [ ]:
sns.displot(height_clean['result_value_median'])

In [ ]:
# the number of unique subject_id should match our original height dataframe
height_clean['subject_id'].nunique() == height['subject_id'].nunique()

# important to check to make sure we didn't accidently lose information

**Important Notes:** 

* The best way to handle these types of inconsistencies will largely depend on the exact variable, question of interest, and domain knowledge. 
<br> </br>
* Seek consultation from mentors and domain experts! 
<br> </br>
* In this case, I'm not considering the date or specific admission. Some patients have multiple admissions. 
<br> </br>
* Additionally, depending how old they are, height may be expected to change over time. In those cases, we might want to consider the patient's age before just taking the median

### Another Example of Data Cleaning (Weight)

In [ ]:
weight.head()

**Review variable types**

It looks like `result_value` is a continuous variable. Let's make sure that Python also has recognized it as such.

In [ ]:
# check variable data types
weight.dtypes

In [ ]:
weight['result_value'] = pd.to_numeric(weight['result_value'])

In [ ]:
# check variable data types
weight.dtypes

In [ ]:
weight['result_value'].describe()

**Distribution of weight values**

In [ ]:
sns.displot(weight['result_value'])

**Investigate the standard deviation in the measurements of patients with multiple observations**

In [ ]:
weight['std'] = weight.groupby('subject_id')['result_value'].transform(np.std, ddof=0)# ddof = 0 indicates that std for patients with one measurement will be displayed as 0 rather than NaN

In [ ]:
weight.sort_values('std', ascending = False)[['subject_id', 'std']].drop_duplicates().head(5)

In [ ]:
weight[weight['subject_id']==10019385].sort_values('chartdate') # did this patient lose > 110 pounds in < 1 month?

**Potential Unit Issue?**

If 1 kg = 2.20462 lbs:

97.30 kg * 2.20462 = 214.5 lbs (this is almost exactly what was measured in the first observation!)

This is a case where I'd feel comfortable manually changing a value - we can feel fairly confident that the 97.3 was measured in kgs instead of lbs

In [ ]:
weight['result_value'] = np.where((weight['subject_id'] == 10019385) & (weight['result_value'] < 100), weight['result_value']*2.20462, weight['result_value']) 

In [ ]:
weight[weight['subject_id']==10019385].sort_values('chartdate') # did this patient lose > 110 pounds in < 1 month?

Review a second patient with high standard deviation

In [ ]:
weight[weight['subject_id']==10021487].sort_values('chartdate')

### Notes:

In the first case, my intuition is that the discrepancy in weight measurements is due to a unit conversions issue

In the second case, the patient has many weight measurements over the a 1.5 year period. The measurements themselves don't vary too widely. 

#### Missing Data

This topic could be it's own class. 

Rule of thumb: Talk to your domain experts/collaborators!

Some common ways to handle missing data:

* Remove those patients
* Impute mean or median
* Forward/backward filling
* MICE

In the above cases, I used the median to impute. This may or may not be the best method depending on your use cases. Here, I wanted to keep it simple!

### Evaluating Unit Conversions

Another clinical variable that often needs cleaning are laboratory test measurements.

In MIMIC we can use the `labevents` and `d_labitems` tables to work with laboratory data. 

Importantly, `d_labitems` links to the `labevents` table. 

The `d_labitems` provides the definitions (or names) or the specific labs via the `itemid`.

In [ ]:
# let's first look at the labevents table
labevents.head()

In [ ]:
# let's first look at the d_labitems table
d_labitems.head()

### Example Data Cleaning (Laboratory Tests)

1. We can merge `d_labitems` and `labevents` by shared identifiers (we'll review this in the next section on merging tables)

2. We can search for specific labs of interest via the `label` column

3. We can also filter with specific `itemid` numbers if we know which correspond to labs of interest

First, let's try and see whether we can find the `itemid` for potassium

In [ ]:
d_labitems[d_labitems['label'].str.contains('potassium', case = False, na=False)]

In [ ]:
potassium = labevents[labevents['itemid']==50971]
potassium.nunique()

In [ ]:
potassium

In [ ]:
sns.displot(potassium['valuenum'])

In [ ]:
# let's check the distinct units of potassium
potassium[['valueuom']].value_counts() # only unit is Milliequivalents per liter 

**Let's evaluate another lab test, one which has more than one unit specification**

In [ ]:
d_labitems[d_labitems['itemid']==51249]

In [ ]:
# *Mean Corpuscular Hemoglobin Concentration
MCHC = labevents[(labevents['itemid']==51249)]
MCHC['valueuom'].value_counts()

In [ ]:
lab_perc = MCHC[MCHC['valueuom']=='%']

In [ ]:
lab_gdl = MCHC[MCHC['valueuom']=='g/dL']

In [ ]:
sns.displot(lab_gdl['valuenum'])

In [ ]:
sns.displot(lab_perc['valuenum'])

### Cleaning EHR Data - Common pitfalls :

- missing data
- inaccurate data
- data inconsistencies
- misunderstanding data and where/how it was generated

## Working with large EHR datasets

#### Merging tables 

Tables can be merged (e.g. linked) together through shared identifiers. The MIMIC official [documentation](https://mimic.mit.edu/docs/iv/) is a great reference for learning more about the associations between tables

First, let's take a look at the diagnosis data that we have for patients

In [ ]:
diagnoses.head()

In order to obtain more detailed information about each ICD code, we can use the the `d_icd_diagnoses` which contains a longer description of each icd code.

In [ ]:
d_icd_diagnoses

This table can be linked to the `diagnoses` dataframe using the `icd_code` and `icd_version` columns

In [ ]:
diagnoses_with_code = pd.merge(diagnoses, 
                               d_icd_diagnoses,
                               on = ['icd_code', 'icd_version'],
                               how = 'inner')

diagnoses_with_code

Let's count the most frequent diagnoses:

In [ ]:
diagnoses_with_code[['long_title']].value_counts().head(10)

Similarly, we can merge the diagnostic data with our `admissions` dataframe in order to aquire more information about the hospital stays of these patients

In [ ]:
admissions_with_icd = pd.merge(admissions,
                               diagnoses_with_code,
                               on = ['subject_id', 'hadm_id'],
                               how = 'inner')

In [ ]:
admissions_with_icd.head()

What if we are interested in a specific disease, but did not know the associated ICD code?

### Filtering datasets (cohort specification)

Let's say we are only interested in patients with sepsis. We could filter our dataset using keyword matching or specific ICD codes (if we have specific codes we are interested in)

In [ ]:
sepsis_cohort = admissions_with_icd[admissions_with_icd['long_title'].str.contains('sepsis', case = False)]

In [ ]:
sepsis_cohort.head()

In [ ]:
sepsis_cohort.nunique()

In [ ]:
sepsis_cohort[['icd_code', 'icd_version', 'long_title']].drop_duplicates()

In [ ]:
sepsis_cohort_icd = admissions_with_icd[admissions_with_icd['icd_code']=='99592']
sepsis_cohort_icd

# Introduction to Machine Learning with scikit-learn

## Case Study: Prediction of ICU Mortality with Supervised Learning

For this demonstration, we will use a previously prepared dataset from an earlier version of MIMIC. This data comes from a [community challenge](https://physionet.org/content/challenge-2012/1.0.0/) focused on predicting mortality with the MIMIC dataset.

We will use the previously pre-processed dataset from one of the challenge winners Alistair Johnson (https://github.com/alistairewj/challenge2012) - so we won't do much of our data cleaning this time (though it is still good practice!). Johnson also includes his [pre-processing notebook](https://github.com/alistairewj/challenge2012/blob/master/prepare-data.ipynb) which can provide another example of how to proceed with cleaning clinical data.

First, we will load in our machine learning functions from scikit-learn

In [ ]:
# load in our machine learning functions from scikit-learn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, roc_auc_score, precision_score, recall_score, accuracy_score, auc, average_precision_score, precision_recall_curve, precision_recall_fscore_support, f1_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV,RandomizedSearchCV,cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In this dataset, we will train a machine learning model to predict mortality using 4,000 unique patient records

In [ ]:
pd.set_option('display.max_columns', None) 

# load in pre-processed challenge dataset
train = pd.read_csv('../challenge2012/data/PhysionetChallenge2012-set-a.csv')
train

This data has already been pre-processed but it is good practice to still examine our data

First, let's examine the variables with the highest number of missing values 

In [ ]:
train.isnull().sum().sort_values(ascending = False)[:10]

Let's create a separate dataframe to tally the number of missing values and plot the proportion of missing values

In [ ]:
# calculate the proportion of missingness
num_missing = train.isnull().sum()
num_missing =  num_missing/len(train)

num_missing.hist()

**Determine which features to keep in our model**

We can see that there are many variables that are largely incomplete. For our purposes, let's first only keep those variables which have <= 25% missing

In [ ]:
vars_low_missing_rate = num_missing[num_missing <= 0.25].index.tolist()

len(vars_low_missing_rate)

Using the list of columns with a low missing rate, we can subset our `train` data to only include these columns 

In [ ]:
# keep columns with a low missing rate
X_vars = train[vars_low_missing_rate]

X_vars.head()

We can further reduce the number of variables by only choosing the first lab or vital measurements

In [ ]:
X_first = X_vars.filter(regex='_first$')# keeps columns with the word 'first'
X_first.columns

We should also consider keeping demographic (`Age`, `Gender`) and baseline information like `Weight` or intensive care unit admitted (`CCU`, `CSRU`, `SICU`) 

Keep Initial Demographics and Clinical Information (Type of ICU)

In [ ]:
X_baseline = X_vars[['Age', 'Gender', 'Weight', 'CCU', 'CSRU', 'SICU']]

Combine maintained set of features

In [ ]:
X_train = pd.merge(X_baseline, 
                   X_first,
                   left_index=True, 
                   right_index=True)

In [ ]:
X_train.head()

Review number of missing variables once more on the 'cleaner' dataset

In [ ]:
X_train.isnull().sum().sort_values()

In [ ]:
# we will drop the 3 patients without a gender
X_train = X_train.dropna(subset=['Gender'])

Create `y` dataframe that will only contain our outcome of interest (`in-hospital_death`)

In [ ]:
# this step takes our `in-hospital_death` column from the original `train` data and combines it with the cleaned up X_train dataframe
# this allows us to remove the 3 patients that we dropped who were missing gender
y = pd.merge(train['In-hospital_death'], X_train,
             left_index=True, 
             right_index=True)

# select outcome of interest
y = y['In-hospital_death']

# rename y df
y_train = y.copy()

Tally the number of patients meeting each outcome

In [ ]:
y_train.value_counts()

### **Training a [Lasso Logistic Regression](https://scikit-learn.org/dev/modules/linear_model.html#binary-case) Model**

These next modules will demonstrate the use of a Least Absolute Shrinkage and Selection Operator (LASSO) logistic regression classifier to predict outcomes

<img src="lecture_images/logistic_regression.png" alt="title" width="400" align="left">

[Image Source](https://www.javatpoint.com/logistic-regression-in-machine-learning)

In logistic regression, we are trying to predict the probability of $y=1$ given a set of features $X$. 

In our case, $y$ represents `in-hospital_death`. Thus, $y = in-hospital__death = 1$ represents that the patient died in the hospital

For each data point $i$ (or each patient observation):

$P(y_i=1|X_i)$

In other words, logistic regresion will allow us to predict the probability that the $y$ outcome of a patient is hospital death given the specific set of $X$ features of that patient.

**Other Notes**:

* Logistic Regeression is used for classification, thus need a binary outcome variable (e.g. survival or death)
* Good baseline model (simple!)
* L1 (Lasso) vs L2 (Ridge) regularization - a penalty to avoid overfitting - essentially, shrinking the coefficients of non-informative variables
* [scikit-learn documentation](https://scikit-learn.org/dev/modules/generated/sklearn.linear_model.LogisticRegression.html#logisticregression) shows us which parameters we can set when constructing a logistic regression model. 

### **Setting up a Machine Learning Pipeline**

Before, we can start training, there is machine learning pre-processing steps we have to take. 

Fortuntately, scikit-learn has a lot of useful functions (such as `Pipepline()`) to help streamline this process. 

The code below sets up the pipeline. Essentially, we want our pipeline to do the following:

1. Impute any missing values (via the median)
2. Standardize numeric variables (mean 0 and standard deviation of 1)

In [ ]:
# specify numeric features to standardize and impute 
numeric_features = ['Age', 'Weight', 'GCS_first', 'Glucose_first', 'HR_first', 'NIDiasABP_first', 'NIMAP_first',
                    'NISysABP_first', 'Temp_first', 'BUN_first', 'Creatinine_first', 'HCO3_first', 'HCT_first', 'K_first', 
                    'Mg_first', 'Na_first', 'PaCO2_first', 'PaO2_first', 'Platelets_first', 'WBC_first', 'pH_first']

# specify non-numeric features
non_numeric_features = ['Gender', 'CCU', 'CSRU', 'SICU']

# construct pipeline for preprocessing data 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

non_numeric_transformer = 'passthrough'  # Pass non-numeric features unchanged

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('non_num', non_numeric_transformer, non_numeric_features)
    ]
)

# Initialize lasso (penalty = 'l1') logistic regression model
# class_weight = 'balanced' since the classes of our model (death vs no death) is heavily imbalanced
lr = LogisticRegression(solver='liblinear', random_state = 12345, penalty = 'l1', class_weight = 'balanced')

# Create a pipeline with preprocessing and a model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lr)  # Replace with your model of choice
])

These pipeline steps make it possible to prepare our data for model training and tuning. 

#### What is model tuning?

Machine learning models have different types of hyperparamters which can be changed to try and optimize model performance. 

We won't discuss too much here, but essentially, we often are training a model to minimize an objective function (also referred to as a loss or cost function depending on the context). Tuning these parameters is what can help minimize the model's objective function and improve classification performance.

Additionally, tuning is often performed using **cross-fold validation**. A technique where a dataset is divived into $k$ folds (usually k=5 or k=10). 

Thus, if $k=5$, we randomly divide the model into 5 equal folds:

<img src="lecture_images/k_fold_cross_validation.png" alt="title" width="400" align="left">

* 4 folds are used for training with different sets of hyperparameters 
* The remaining held-out fold is used how the model does on data it hasn't seen before
* This process is repeated until each fold is used as the held-out test set once
* The optimal hyperparaters are chosen based on the average performance across folds

#### How do we choose and evaluate the different hyperparameters?

In this demonstration, we will evaluate hyperparameters using the scikit-learn function `GridSearch` - this allows us to pre-specify the hyperparameters we want to evaluate in a grid. During Cross-fold validation, we we test all possible hyperparameter combinations in this grid to determine those which provide the best performance across folds

### Cross-validation and GridSearch

Let's set up `GridSearchCV` framework to train and tune our model with 5-fold Cross Validation

In [ ]:
# Define parameter grid for model tuning
param_grid = {
    'classifier__tol': [1e-4, 1e-3, 1e-2, 1e-1],
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]   # Range of regularization strengths (inverse of regularization strength)
}

# Set up GridSearchCV
lr_grid_search = GridSearchCV(estimator=model_pipeline, # specific our pipeline as our estimator 
                              param_grid=param_grid,
                              cv=5,
                              scoring='roc_auc', # this specifies that we are going to use the AUC as a metric to evaluate how well our model does across folds
                              return_train_score=True,
                              refit=True,
                              n_jobs=-1)

Now that we've specified our pipeline and implemented it into our gridsearch/cross validation, we can train (or fit!) our model to our training data. The code will automatically create our folds for us!

In [ ]:
# Fit the random search object to the data
fit_lr_grid_search = lr_grid_search.fit(X_train, y_train)
fit_lr_grid_search

#### So what is going on when we fit this model?

Our pipeline ensures the following:

1. Our data is divided into 5-folds
2. We train the model on 4-folds
3. These 4-folds are pre-processed (numeric data standardized and imputed as needed)
4. GridSearch tests each combination of hyperparamters (`tol=1e-4` and `C=0.001`; `tol=1e-4` and `C=0.01`; and so forth)
5. For each fold and combination of hyperparameters, the model calculates the AUC on the held-out fold
6. This process repeats until all 5-folds and hyperparamter combos have been evaluted
7. The set of hyperparameters with the highest average AUC will be chosen as our best model

#### **Concept Check:** 

Why do we perform the pre-processing step on 4-folds at a time? Can we standardize and impute all of the data prior to starting the training/tuning steps? Why or why not?

### Model Evaluation on Training set

We can determine what the best hyperparameters and the average AUC is from the fitted object `fit_lr_grid_search`:

In [ ]:
# Best parameters and score
print(f"Best performing hyperparameters: {fit_lr_grid_search.best_params_}")
print(f"Average AUC: {fit_lr_grid_search.best_score_.round(4)}")

We can also see the results from all of the models evaluated during the tuning process

In [ ]:
# Access the cv_results_ dictionary
cv_results = fit_lr_grid_search.cv_results_

# Create a DataFrame from cv_results_
results_df = pd.DataFrame(cv_results)

results_df

Let's just look at the best performing model (based on AUC)

In [ ]:
# Extract the row corresponding to the best index
best_row_df = results_df.loc[[fit_lr_grid_search.best_index_]]  # Use double brackets to keep it as a DataFrame
best_row_df

## Training Evaluation

There are many ways to evaluate the performance of our training. The below modules will review a few!

### Model Coefficients (important predictors)

Default coefficients are represented as log-odds

We can exponentiate the log-odds to get the odds ratio:

$\text{Odds Ratio} = e^{\text{coef}}$

**To intepret:**
* A feature with an odds ratio > 1 - are features in which the odds of death increase when that feature increases
* A feature with an odds ratio < 1 - are features in which the odds of death decrease when that feature increases
* A feature with an odds ratio = 1 - are features which have no effect on the outcome (death)



In [ ]:
# Access the best estimator
best_lasso = lr_grid_search.best_estimator_

# Extract coefficients from the Lasso model
lasso_coefficients = best_lasso.named_steps['classifier'].coef_.flatten().tolist()

# If needed, map coefficients to feature names
feature_names = X_train.columns.tolist()  # Replace with your list of feature names
coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lasso_coefficients
})

coefficients_df['abs(coef)'] = coefficients_df['Coefficient'].abs()
coefficients_df['Odds Ratio'] = np.exp(coefficients_df['Coefficient'])

# Display the coefficients
coefficients_df.sort_values(by='Odds Ratio', ascending=False).reset_index(drop=True).head(5)

### Confusion Matrix

A confusion matrix helps us calculate our true and false positive rates. This in turn, helps us calculate important metrics such as: 

* sensitivity - TP / (TP + FN)
* specificity - TN / (TN + FP) 
* positive predictive value (precision) - TP / (TP + FP)
* negative predictive value - TN / (TN + FN)

In [ ]:
# predicted classes using default 0.5 threshold
y_hat = fit_lr_grid_search.predict(X_train) 

print("classification report:\n ", classification_report(y_train, y_hat, digits=3))
print("confusion matrix:\n ")

# Create the confusion matrix
cm = confusion_matrix(y_train, y_hat)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

### Receiver Operating Characteristic Curve

A ROC plots the true postitve rate (sensitivity) vs false positive rate (1-specificity) at different decision thresholds. This allows us to calculate the area under the curve (AUC) to estimate model performance. An AUC of 0.5 indicates a model that performs as well as a coin flip (50/50).

The use of the `GridSearchCV()` function, automatically refits the best trained/tuned model using all of our data.

Thus, we can use the fitted object to evaluate some performance metrics

In [ ]:
# calculate the predicted probabilities on all of the training data
y_proba = fit_lr_grid_search.predict_proba(X_train)[:,1] 

# calculate the area under the curve (AUC)
auc = roc_auc_score(y_train, y_proba)

In [ ]:
# calculate false positive rate and the true positive rate in order to construct a receiver operating curve (ROC)
fpr, tpr, thresholds = roc_curve(y_train, y_proba, pos_label=1)

plt.title('ROC (Train)')
ax1 = plt.plot(fpr, tpr, 'b', label = '%s AUC = %0.4f' % ('', auc), linewidth=2)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Precision-Recall Curve

The curve plots the prcesion and recall at different decision thresholds. Recall or the true positive rate tells us how well the model correctly identifies all positive cases, whereas precision estimates the proportion of cases the model identified as positive that were truely positive.

In [ ]:
y_proba = fit_lr_grid_search.predict_proba(X_train)[:,1]

precision, recall, thresholds = precision_recall_curve(y_train, y_proba, pos_label=1, sample_weight=None)
avg_p = average_precision_score(y_train, y_proba, pos_label=1, sample_weight=None)

plt.title('Precision-Recall Curve (Train)')
ax1= plt.plot(precision, recall, 'b', label = '%s AP = %0.3f' % ('', avg_p), linewidth=2)
plt.legend(loc = 'lower left')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Precision')
plt.xlabel('Recall')

## Evaluation on Held-out Test Set

Let's see how well our trained model does on data it has never seen before

In [ ]:
test = pd.read_csv('../challenge2012/data/PhysionetChallenge2012-set-b.csv')
test

We have to make sure our data is structured simiarly 

In [ ]:
X_test = test[['Age', 'Gender', 'Weight', 'CCU', 'CSRU', 'SICU', 'GCS_first',
       'Glucose_first', 'HR_first', 'NIDiasABP_first', 'NIMAP_first',
       'NISysABP_first', 'Temp_first', 'BUN_first', 'Creatinine_first',
       'HCO3_first', 'HCT_first', 'K_first', 'Mg_first', 'Na_first',
       'PaCO2_first', 'PaO2_first', 'Platelets_first', 'WBC_first',
       'pH_first']]

# we will drop the small handful of patients without a gender
X_test = X_test.dropna(subset=['Gender'])

## format the y_test variable
y_test = test.copy()

# merge with the index for train to remove the handful of patients without gender
y_test = pd.merge(y_test['In-hospital_death'], X_test,
                  left_index=True, 
                  right_index=True)

# select only the outcome of interest
y_test = y_test['In-hospital_death']

### Final Test-set Evaluation Metrics

#### Confusion Matrix (Test)

In [ ]:
# predicted classes using default 0.5 threshold
y_hat_test = fit_lr_grid_search.predict(X_test) 

print("classification report (Test):\n ", classification_report(y_test, y_hat_test, digits=3))

# Create the confusion matrix
cm = confusion_matrix(y_test, y_hat_test)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

#### Receiver Operating Characteristic Curve (Test)

In [ ]:
# calculate the predicted probabilities on all of the training data
y_proba_test = fit_lr_grid_search.predict_proba(X_test)[:,1] 

# calculate the area under the curve (AUC)
auc = roc_auc_score(y_test, y_proba_test)

In [ ]:
# calculate false positive rate and the true positive rate in order to construct a receiver operating curve (ROC)
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_proba_test, pos_label=1)

plt.title('ROC (Test)')
ax1 = plt.plot(fpr_test, tpr_test, 'b', label = '%s AUC = %0.4f' % ('', auc), linewidth=2)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

#### Precision-Recall Curve (Test)


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_test, pos_label=1, sample_weight=None)
avg_p_test = average_precision_score(y_test, y_proba_test, pos_label=1, sample_weight=None)

plt.title('Precision-Recall Curve (Test)')
ax1= plt.plot(precision, recall, 'b', label = '%s AP = %0.3f' % ('', avg_p_test), linewidth=2)
plt.legend(loc = 'lower left')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Precision')
plt.xlabel('Recall')

### **Model Evaluation on held-out test-set**

- metrics for evaluation

- think about the clinical outcome of interest - what evaluation metric makes the most sense 

### **Error Analysis for Model Diagnosis**

- which patients are we getting wrong/confusion matrix

#### **Questions to Consider:**

* Which features seem most predictive for classification? How do you know?

* What metrics should we prioritize when evaluating this model?

* How might the following affect model performance:

    * Missing values

    * Class imbalance
    
* What other types of experiments could we perform with this dataset? Is mortality prediction the best use case? Why or why not?

* Are there any downsides to grid search? What might be some other ways to train/tune the model?

## **Additional Experiments**

---

**What happens if we train our model on a smaller dataset of only 500 patients with **X** having the positive class?**


**Rather than imputing, how does model performance change if we drop all patients with at least 1 missing value?**

- Could there be any biases when we drop patients with 'missing' values

**Does the model perform better if we specify patients who have missing values (i.e. create an additional column)**

## Random Forest Algorithm

<img src="lecture_images/decision_tree.png" alt="title" width="400" align="left">

* Random forest is an esembling algorithm that learns from generating multiple (hundreds-thousands) of decision trees
* Randomly selects a subset of features when growing each tree 
* This approach allows us to average the predictions of the "crowd"

<b></b>
* Many **hyperparameters** that can be tuned during training:
<br></br>
* `n_estimators`: Number of trees
* `max_features`: Number of featues to be considered at each split 
* `max_depth`: Maximum number of levels in tree (can help regularize the tree to prevent overfitting)
* `min_samples_split`: Minimum samples to split a node 
* `min_samples_leaf`: Minimum number of samples required at each leaf node 

In [ ]:
### tuning RF hyperparameters
# Number of trees in random forest
n_estimators = [100, 300, 500, 1000]
# Number of features to consider at every split
max_features = [2, 5]
# Maximum number of levels in tree
max_depth = [5, 8]
# Minimum number of samples required to split a node
min_samples_split = [5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10, 15]

# create grid of hypterparameter settings
# we will permute through this grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

### Hyperparameter Tuning

In [ ]:
# Create a random forest classifier
rf = RandomForestClassifier(criterion='entropy', random_state=12345, class_weight = 'balanced')

# Use random search to find the best hyperparameters
grid_search = GridSearchCV(estimator = rf,
                           param_grid = param_grid,
                           cv = 5,
                           scoring = 'roc_auc',
                           return_train_score = False,
                           n_jobs = -1)

# Fit the random search object to the data
grid_search.fit(X_train, y_train)

In [ ]:
# Create a variable for the best model
best_rf = grid_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  grid_search.best_params_)

In [ ]:
# Access the cv_results_ dictionary
cv_results_rf = grid_search.cv_results_

# Create a DataFrame from cv_results_
rf_results_df = pd.DataFrame(cv_results_rf)

# Extract the row corresponding to the best index
rf_best_row_df = rf_results_df.loc[[grid_search.best_index_]]  # Use double brackets to keep it as a DataFrame
rf_best_row_df

### Training Evaluation

In [ ]:
y_pred = best_rf.predict(X_train)
y_prob = best_rf.predict_proba(X_train)[:,1]

accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
auroc = roc_auc_score(y_train, y_prob)

print("Accuracy:", accuracy)
print("AUC:", auroc)
print("Precision:", precision)
print("Recall:", recall)

### Feature Importance

In [ ]:
# review feature importances on the training set
feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

# Plot a simple bar chart
feature_importances.plot.bar();

In [ ]:
y_hat = best_rf.predict(X_train) # predicted classes using default 0.5 threshold
y_proba = best_rf.predict_proba(X_train)[:,1] #predicted probabilities
auc=roc_auc_score(y_train, y_proba)
loss= log_loss(y_train, y_hat)

print ('the AUC is: {:0.3f}'.format(auc))
print ('the logloss is: {:0.3f}'.format(loss))
print("classification report:\n ", classification_report(y_train,y_hat, digits=3))
print("confusion matrix:\n ")
# Create the confusion matrix
cm = confusion_matrix(y_train, y_hat)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

### Receiver Operating Characteristic Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train, y_proba, pos_label=1)

plt.title('ROC curve')
ax1 = plt.plot(fpr, tpr, 'b', label = '%s AUC = %0.3f' % ('', auc), linewidth=2)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Precision-Recall Curve

In [ ]:
y_proba = best_rf.predict_proba(X_train)[:,1]

precision, recall, thresholds = precision_recall_curve(y_train, y_proba, pos_label=1, sample_weight=None)
avg_p = average_precision_score(y_train, y_proba, pos_label=1, sample_weight=None)

plt.title('Precision-Recall curve')
ax1= plt.plot(precision, recall, 'b', label = '%s AP = %0.3f' % ('', avg_p), linewidth=2)
plt.legend(loc = 'lower left')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Precision')
plt.xlabel('Recall')

### Need to Review

### Evaluation of Test Set

In [ ]:
y_hat = best_rf.predict(X_test) # predicted classes using default 0.5 threshold
y_proba = best_rf.predict_proba(X_test)[:,1] #predicted probabilities
auc=roc_auc_score(y_test, y_proba)
loss= log_loss(y_test, y_hat)

print ('the AUC is: {:0.3f}'.format(auc))
print ('the logloss is: {:0.3f}'.format(loss))
print("classification report:\n ", classification_report(y_test,y_hat, digits=3))
print("confusion matrix:\n ")
# Create the confusion matrix
cm = confusion_matrix(y_test, y_hat)
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

### Receiver Operating Characteristic Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_proba, pos_label=1)

plt.title('ROC curve')
ax1 = plt.plot(fpr, tpr, 'b', label = '%s AUC = %0.3f' % ('', auc), linewidth=2)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Precision-Recall Curve

In [ ]:
y_proba = best_rf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_proba, pos_label=1, sample_weight=None)
avg_p = average_precision_score(y_test, y_proba, pos_label=1, sample_weight=None)

plt.title('Precision-Recall curve')
ax1= plt.plot(precision, recall, 'b', label = '%s AP = %0.3f' % ('', avg_p), linewidth=2)
plt.legend(loc = 'lower left')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Precision')
plt.xlabel('Recall')